In [28]:
from Bio import SeqIO
from pandas import DataFrame
import pandas as pd
import os, sys
from tqdm import tqdm
import zipfile

In [36]:
def BGC_counter(refseq) :
    gbk_file = refseq
    print (gbk_file)
    df_antismash = DataFrame(columns=("Cluster No","Cluster Type"))
    for seq_record in SeqIO.parse(gbk_file, "genbank") :
        for seq_feat in seq_record.features :
            if seq_feat.type == "cluster" :
                cluster_number = seq_feat.qualifiers["note"][0].replace(" ","_").replace(":","")
                cluster_type = seq_feat.qualifiers["product"][0]
                df = pd.DataFrame([[cluster_number, cluster_type]], columns=("Cluster No", "Cluster Type"))
                df_antismash = df_antismash.append(df, ignore_index=True)
    return df_antismash

In [41]:
os.chdir("C:/Users/Ahmad Ardi/")
cwd = os.getcwd()
id = pd.read_csv('02_NCBI_Result_WGS.csv')
for i in tqdm(range(len(id))):    
    try:
        genus = id.loc[i, "Genus"]
        species = id.loc[i, "Species"]
        refseq = pd.read_csv('Refseq_'+genus+'_'+species+'.csv')
        os.chdir(cwd+"/Result/"+genus+" "+species)
        #_______________________________________________________
        cwd2 = (os.getcwd())
        result = os.listdir()
        for x in tqdm(range(len(result))):
            if '.zip' in result[x]:
                print(result[x])
                with zipfile.ZipFile(result[x]) as zip_ref:
                    zip_ref.extractall()
        for z in range(len(result)):
            if 'bacteria' in result[z]:
                #print (result[z])
                os.chdir(result[z]+"/")
                #___________________________________________________
                #print(os.getcwd())
                dir_ref = os.listdir()
                #print(dir_ref)
                for y in range(len(dir_ref)):
                    #print (dir_ref[y])
                    if 'final.gbk' in dir_ref[y]:
                        df = BGC_counter(dir_ref[y])
                        #print(df)
                        os.chdir(cwd2)
                        #____________________________________________
                        df.to_csv("BGC_count_"+dir_ref[y].replace('.final.gbk','')+".csv")
                        #print(os.getcwd())
                        print('_____________________________________________')
                os.chdir(cwd2)
                print(os.getcwd())
            os.chdir(cwd2)
        os.chdir(cwd)
    except OSError:
        print('ERROR_'+genus+' '+species)
        os.chdir(cwd)
        #pass

  0%|          | 0/10 [00:00<?, ?it/s]

ERROR_Cutibacterium acnes
ERROR_Staphylococcus epidermidis
ERROR_Corynebacterium tuberculostearicum
ERROR_Staphylococcus capitis
ERROR_Corynebacterium simulans
ERROR_Streptococcus mitis
ERROR_Staphylococcus hominis
ERROR_Corynebacterium aurimucosum
ERROR_Corynebacterium kroppenstedtii
ERROR_Corynebacterium amycolatum


100%|██████████| 10/10 [00:00<00:00, 242.30it/s]


In [34]:
os.chdir('C:/Users/Ahmad Ardi')
df_count = DataFrame(columns=("Species", "Strain"))

In [35]:
df_count = DataFrame(columns=("Species", "Strain"))
cwd = os.getcwd()
species = pd.read_csv('02_NCBI_Result_WGS.csv')
for i in tqdm(range(len(species))):
    try:
        refseq = pd.read_csv('Refseq_'+species.loc[i,"Genus"]+'_'+species.loc[i,"Species"]+'.csv')
        #print(refseq)
        os.chdir(cwd+"/Result/"+species.loc[i,"Genus"]+" "+species.loc[i,"Species"]+"/")
        print(os.getcwd())
        result = os.listdir()
        for x in range(len(result)):
            if 'BGC_count_' in result[x]:
                print(result[x])
                df = pd.read_csv(result[x])
                df2 = df["Cluster Type"].value_counts()
                acc = result[x]
                acc = acc.replace("BGC_count_","")
                acc = acc.replace(".csv","")
                strain = refseq.loc[refseq["Refseq"] == acc, "Strain"]
                df2 = df2.rename(acc)
                df_count = df_count.append(df2)
                df_count.loc[acc, "Species"] = species.loc[i,"Genus"]+" "+species.loc[i,"Species"]
                try:
                    df_count.loc[acc, "Strain"] = strain.values[0]
                except IndexError:
                    pass
                c = df_count.reset_index()
        os.chdir(cwd)
        #else:
            #os.chdir(cwd)
    except OSError:  
        #os.chdir(cwd)
        print('error')
        pass

  0%|          | 0/10 [00:00<?, ?it/s]

error
error
error
error
error
error
error
error
error
error


100%|██████████| 10/10 [00:00<00:00, 464.94it/s]


In [31]:
c = c.fillna(0)
c.to_csv("03_BGC_Result.csv")
c

NameError: name 'c' is not defined

In [10]:
import seaborn as sns; sns.set()
import numpy as np

df3 = c.drop(columns=["index", "Strain"])
df3 = df3.set_index('Species')
df3

NameError: name 'c' is not defined

In [11]:
%matplotlib inline
g = sns.heatmap(df3, annot=True, cbar_kws={"orientation":"vertical"})
g.set_xticklabels(g.get_xticklabels(), rotation = 45)
g.xaxis.set_ticks_position('top')
g.xaxis.set_label_position('top')
g.set_xlabel('Biosynthetic Gene Clusters')

NameError: name 'df3' is not defined

In [12]:
ax = df3.plot(kind='barh', stacked=True, legend=False, figsize=(10,5), colormap="Accent")
ax.set_xlabel("Biosynthetic Gene Clusters")
chartBox = ax.get_position()
ax.set_position([chartBox.x0, chartBox.y0, chartBox.width*0.6, chartBox.height])
ax.legend(loc='upper right', bbox_to_anchor=(1.45, 0.8), shadow=True, ncol=1)

NameError: name 'df3' is not defined

In [13]:
df4 = df3.groupby(level=0).mean()
df4

NameError: name 'df3' is not defined

In [15]:
%matplotlib inline
ax = df4.plot(kind='barh', stacked=True, legend=False, figsize=(10,3), colormap=Accent)
ax.set_xlabel("Biosynthetic Gene Clusters")
#patches, labels = ax.get_legend_handles_labels()
#ax.legend(loc=9, bbox_to_anchor=(0.5, -0.1), ncol=3)
chartBox = ax.get_position()
ax.set_position([chartBox.x0, chartBox.y0, chartBox.width*0.6, chartBox.height])
ax.legend(loc='upper right', bbox_to_anchor=(1.45, 0.8), shadow=True, ncol=1)
plt.savefig('BGCs.png', bbox_inches='tight')

NameError: name 'df4' is not defined